In [63]:
import sys
sys.path.insert(0, '../ELINA/python_interface/')

import numpy as np
import re
import csv
from elina_box import *
from elina_interval import *
from elina_abstract0 import *
from elina_manager import *
from elina_dimension import *
from elina_scalar import *
from elina_interval import *
from elina_linexpr0 import *
from elina_lincons0 import *
import ctypes
from ctypes.util import find_library
from gurobipy import *
import time

In [64]:
# Import for debugging in jupyter notebook
from IPython.core.debugger import set_trace #TODO remove at end.

In [65]:
libc = CDLL(find_library('c'))
cstdout = c_void_p.in_dll(libc, 'stdout')

In [66]:
class layers:
    def __init__(self):
        self.layertypes = []
        self.weights = []
        self.biases = []
        self.numlayer = 0
        self.ffn_counter = 0
        self.rank = []
        self.use_LP = []

In [67]:
def parse_bias(text):
    if len(text) < 1 or text[0] != '[':
        raise Exception("expected '['")
    if text[-1] != ']':
        raise Exception("expected ']'")
    v = np.array([*map(lambda x: np.double(x.strip()), text[1:-1].split(','))])
    #return v.reshape((v.size,1))
    return v

In [68]:
def parse_vector(text):
    if len(text) < 1 or text[0] != '[':
        raise Exception("expected '['")
    if text[-1] != ']':
        raise Exception("expected ']'")
    v = np.array([*map(lambda x: np.double(x.strip()), text[1:-1].split(','))])
    return v.reshape((v.size,1))
    #return v

In [69]:
def balanced_split(text):
    i = 0
    bal = 0
    start = 0
    result = []
    while i < len(text):
        if text[i] == '[':
            bal += 1
        elif text[i] == ']':
            bal -= 1
        elif text[i] == ',' and bal == 0:
            result.append(text[start:i])
            start = i+1
        i += 1
    if start < i:
        result.append(text[start:i])
    return result

In [70]:
def parse_matrix(text):
    i = 0
    if len(text) < 1 or text[0] != '[':
        raise Exception("expected '['")
    if text[-1] != ']':
        raise Exception("expected ']'")
    return np.array([*map(lambda x: parse_vector(x.strip()).flatten(), balanced_split(text[1:-1]))])

In [71]:
def parse_net(text):
    lines = [*filter(lambda x: len(x) != 0, text.split('\n'))]
    i = 0
    res = layers()
    while i < len(lines):
        if lines[i] in ['ReLU', 'Affine']:
            W = parse_matrix(lines[i+1])
            b = parse_bias(lines[i+2])
            res.layertypes.append(lines[i])
            res.weights.append(W)
            res.biases.append(b)
            res.numlayer+= 1
            res.rank.append(np.zeros((W.shape[0],1)))
            res.use_LP.append(np.full((W.shape[0],1), False))
            i += 3
        else:
            raise Exception('parse error: '+lines[i])
    return res

In [72]:
def parse_spec(text):
    text = text.replace("[", "")
    text = text.replace("]", "")
    with open('dummy', 'w') as my_file:
        my_file.write(text)
    data = np.genfromtxt('dummy', delimiter=',',dtype=np.double)
    low = np.copy(data[:,0])
    high = np.copy(data[:,1])
    return low,high

In [73]:
def get_perturbed_image(x, epsilon):
    image = x[1:len(x)]
    num_pixels = len(image)
    LB_N0 = image - epsilon
    UB_N0 = image + epsilon
     
    for i in range(num_pixels):
        if(LB_N0[i] < 0):
            LB_N0[i] = 0
        if(UB_N0[i] > 1):
            UB_N0[i] = 1
    return LB_N0, UB_N0

In [74]:
def generate_linexpr0(weights, bias, size):
    linexpr0 = elina_linexpr0_alloc(ElinaLinexprDiscr.ELINA_LINEXPR_DENSE, size)
    cst = pointer(linexpr0.contents.cst)
    elina_scalar_set_double(cst.contents.val.scalar, bias)
    for i in range(size):
        elina_linexpr0_set_coeff_scalar_double(linexpr0,i,weights[i])
    return linexpr0

In [75]:
def analyze(nn, LB_N0, UB_N0, label):   
    num_pixels = len(LB_N0)
    nn.ffn_counter = 0
    numlayer = nn.numlayer 
    man = elina_box_manager_alloc()
    itv = elina_interval_array_alloc(num_pixels)
    for i in range(num_pixels):
        elina_interval_set_double(itv[i],LB_N0[i],UB_N0[i])

    ## construct input abstraction
    element = elina_abstract0_of_box(man, 0, num_pixels, itv)
    elina_interval_array_free(itv,num_pixels)
    for layerno in range(numlayer):
        if(nn.layertypes[layerno] in ['ReLU', 'Affine']):
            weights = nn.weights[nn.ffn_counter]
            biases = nn.biases[nn.ffn_counter]
            dims = elina_abstract0_dimension(man,element)
            num_in_pixels = dims.intdim + dims.realdim
            num_out_pixels = len(weights)

            dimadd = elina_dimchange_alloc(0,num_out_pixels)    
            for i in range(num_out_pixels):
                dimadd.contents.dim[i] = num_in_pixels
            elina_abstract0_add_dimensions(man, True, element, dimadd, False)
            elina_dimchange_free(dimadd)
            np.ascontiguousarray(weights, dtype=np.double)
            np.ascontiguousarray(biases, dtype=np.double)
            var = num_in_pixels
            # handle affine layer
            for i in range(num_out_pixels):
                tdim= ElinaDim(var)
                linexpr0 = generate_linexpr0(weights[i],biases[i],num_in_pixels)
                element = elina_abstract0_assign_linexpr_array(man, True, element, tdim, linexpr0, 1, None)
                var+=1
            dimrem = elina_dimchange_alloc(0,num_in_pixels)
            for i in range(num_in_pixels):
                dimrem.contents.dim[i] = i
            elina_abstract0_remove_dimensions(man, True, element, dimrem)
            elina_dimchange_free(dimrem)
            # handle ReLU layer 
            if(nn.layertypes[layerno]=='ReLU'):
                element = relu_box_layerwise(man,True,element,0, num_out_pixels)
            nn.ffn_counter+=1 

        else:
            print(' net type not supported')
   
    dims = elina_abstract0_dimension(man,element)
    output_size = dims.intdim + dims.realdim
    # get bounds for each output neuron
    bounds = elina_abstract0_to_box(man,element)

           
    # if epsilon is zero, try to classify else verify robustness 
    
    verified_flag = True
    predicted_label = 0
    if(LB_N0[0]==UB_N0[0]):
        for i in range(output_size):
            inf = bounds[i].contents.inf.contents.val.dbl
            flag = True
            for j in range(output_size):
                if(j!=i):
                    sup = bounds[j].contents.sup.contents.val.dbl
                    if(inf<=sup):
                        flag = False
                        break
            if(flag):
                predicted_label = i
                break    
    else:
        inf = bounds[label].contents.inf.contents.val.dbl
        for j in range(output_size):
            if(j!=label):
                sup = bounds[j].contents.sup.contents.val.dbl
                if(inf<=sup):
                    predicted_label = label
                    verified_flag = Falselse
                    break

    elina_interval_array_free(bounds,output_size)
    elina_abstract0_free(man,element)
    elina_manager_free(man)        
    return predicted_label, verified_flag

In [76]:
# def main(netname, specname, esilon, c_label = None, method = 'box'):
#     with open(netname, 'r') as netfile:
#         netstring = netfile.read()
#     with open(specname, 'r') as specfile:
#         specstring = specfile.read()
#     nn = parse_net(netstring)
#     x0_low, x0_high = parse_spec(specstring)
#     LB_N0, UB_N0 = get_perturbed_image(x0_low,0)
    
#     label, _ = analyze(nn,LB_N0,UB_N0,0) # Get label of unperturbed image, i.e. eps=0
#     start = time.time()
#     if method == 'box':
#         if(label==int(x0_low[0])):
#             LB_N0, UB_N0 = get_perturbed_image(x0_low,epsilon)
#             _, verified_flag = analyze_box(nn,LB_N0,UB_N0,label)
#             if(verified_flag):
#                 print("verified")
#             else:
#                 print("can not be verified")  
#         else:
#             print("image not correctly classified by the network. expected label ",int(x0_low[0]), " classified label: ", label)
#     if method == 'linear':
#         if(label==int(x0_low[0])):
#             LB_N0, UB_N0 = get_perturbed_image(x0_low,epsilon)
#             _, verified_flag = analyze_linear(nn,LB_N0,UB_N0,label)
#             if(verified_flag):
#                 print("verified")
#             else:
#                 print("can not be verified")  
#         else:
#             print("image not correctly classified by the network. expected label ",int(x0_low[0]), " classified label: ", label)
#     end = time.time()
#     print("analysis time: ", (end-start), " seconds")
    

In [77]:
# def compute_rank(nn, rank_threshold, norm=1, skip_first_layer=True, skip_last_layer=True):
#     numlayer = nn.numlayer 
#     norms = np.zeros((nn.weights[1].shape[0], numlayer-2)) # TODO use list or similar if first and last layer are also used
#     for layerno in range(numlayer):
#         if skip_first_layer and layerno == 0:
#             #TODO treat first layer
#             continue
#         if skip_last_layer and layerno == numlayer-1:
#             continue
#         if(nn.layertypes[layerno] in ['ReLU', 'Affine']):
#             weights = nn.weights[layerno]
#             biases = nn.biases[layerno]
#             norms[:, layerno-1] = np.linalg.norm(weights, ord=norm, axis=1) + biases
#         else:
#             print(' net type not supported')
    
#     rank_idxs = np.argsort(-norms, axis=None)
#     rank_idxs = rank_idxs.reshape(nn.weights[1].shape[0], int(len(rank_idxs)/ nn.weights[1].shape[0]))
    
#     for layerno in range(numlayer):
#         if skip_first_layer and layerno == 0:
#             continue
#         if skip_last_layer and layerno == numlayer-1:
#             continue
#         if(nn.layertypes[layerno] in ['ReLU', 'Affine']):
#             nn.rank[layerno] = rank_idxs[:, layerno-1]
#             nn.use_LP[layerno] = nn.rank[layerno]  < rank_threshold
#         else:
#             print(' net type not supported')

## Define operations on abstract domain using Box approximations

In [78]:
def get_relu_hidden_bounds_using_box(man, weights, biases, input_LB, input_UB, num_in_pixels):
    '''
    This function calculates the bounds of a ReLU operation followed by a hidden layer. 
    INPUT:
        - man: pointer to elina manager
        - weights: weights of the hidden layer
        - biases: biases of the hidden layer
        - input_LB: lower bound of the inputs to the ReLU
        - input_UB: upper bound of the inputs to the ReLU
        - num_in_pixels: number of inputs to ReLU
    
    OUTPUT:
        - output_LB: lower bound of the outputs from hidden layer
        - output_UB: upper bound of the outputs from hidden layer
        - num_out_pixels: number of outputs of hidden layer
    '''
    itv = elina_interval_array_alloc(num_in_pixels)

    ## Populate the interval
    for i in range(num_in_pixels):
        elina_interval_set_double(itv[i], input_LB[i], input_UB[i])

    ## construct input abstraction
    element = elina_abstract0_of_box(man, 0, num_in_pixels, itv)
    elina_interval_array_free(itv, num_in_pixels)
    
    # ------------------------------------------------------------------
    # Handle ReLU Layer
    # ------------------------------------------------------------------

    element = relu_box_layerwise(man, True, element,0, num_in_pixels)
    
    # ------------------------------------------------------------------
    # Handle Affine Layer
    # ------------------------------------------------------------------

    # calculate number of outputs
    num_out_pixels = len(weights)
    
    print("[Network] Input pixels: " + str(num_in_pixels))
    print("[Network] Shape of weights: " + str(np.shape(weights)))
    print("[Network] Shape of biases: " + str(np.shape(biases)))
    print("[Network] Out pixels: " + str(num_out_pixels))

    # Create number of neurons in the layer and populate it
    # with the number of inputs to each neuron in the layer
    dimadd = elina_dimchange_alloc(0, num_out_pixels)    
    for i in range(num_out_pixels):
        dimadd.contents.dim[i] = num_in_pixels

    # Add dimensions to an ElinaAbstract0 pointer i.e. element
    elina_abstract0_add_dimensions(man, True, element, dimadd, False)
    elina_dimchange_free(dimadd)

    # Create the linear expression associated each neuron
    var = num_in_pixels
    for i in range(num_out_pixels):
        tdim = ElinaDim(var)
        linexpr0 = generate_linexpr0(weights[i], biases[i], num_in_pixels)
        # Parallel assignment of several dimensions of an ElinaAbstract0 by using an ElinaLinexpr0Array
        element = elina_abstract0_assign_linexpr_array(man, True, element, tdim, linexpr0, 1, None)
        var += 1

    # Pointer to which semantics we want to follow.
    dimrem = elina_dimchange_alloc(0, num_in_pixels)
    for i in range(num_in_pixels):
        dimrem.contents.dim[i] = i
        
    # Remove dimensions from an ElinaAbstract0
    elina_abstract0_remove_dimensions(man, True, element, dimrem)
    elina_dimchange_free(dimrem)
    
    # get bounds for each output neuron
    bounds = elina_abstract0_to_box(man,element)
    
    output_LB = np.zeros((num_out_pixels, 1), float)
    output_UB = np.zeros((num_out_pixels, 1), float)
    for j in range(num_out_pixels):
        output_LB[j] = bounds[j].contents.inf.contents.val.dbl
        output_UB[j] = bounds[j].contents.sup.contents.val.dbl
    
    # free out the memory allocations
    elina_interval_array_free(bounds, num_out_pixels)
    elina_abstract0_free(man, element)
    
    return output_LB, output_UB, num_out_pixels

In [79]:
def get_relu_bounds_using_box(man, input_LB, input_UB, num_in_pixels):
    '''
    This function calculates the bounds of a ReLU operation. 
    INPUT:
        - man: pointer to elina manager
        - input_LB: lower bound of the inputs to the ReLU
        - input_UB: upper bound of the inputs to the ReLU
        - num_in_pixels: number of inputs to ReLU
    
    OUTPUT:
        - output_LB: lower bound of the outputs from ReLU layer
        - output_UB: upper bound of the outputs from ReLU layer
        - num_out_pixels: number of outputs of ReLI layer
    '''
    itv = elina_interval_array_alloc(num_in_pixels)

    ## Populate the interval
    for i in range(num_in_pixels):
        elina_interval_set_double(itv[i], input_LB[i], input_UB[i])

    ## construct input abstraction
    element = elina_abstract0_of_box(man, 0, num_in_pixels, itv)
    elina_interval_array_free(itv, num_in_pixels)
    
    # ------------------------------------------------------------------
    # Handle ReLU Layer
    # ------------------------------------------------------------------
    num_out_pixels = num_in_pixels
    
    element = relu_box_layerwise(man, True, element,0, num_in_pixels)
    
    # get bounds for each output neuron
    bounds = elina_abstract0_to_box(man,element)
    
    # get bounds for each output neuron
    bounds = elina_abstract0_to_box(man,element)
    
    output_LB = np.zeros((num_out_pixels, 1), float)
    output_UB = np.zeros((num_out_pixels, 1), float)
    for j in range(num_out_pixels):
        output_LB[j] = bounds[j].contents.inf.contents.val.dbl
        output_UB[j] = bounds[j].contents.sup.contents.val.dbl
    
    # free out the memory allocations
    elina_interval_array_free(bounds, num_out_pixels)
    elina_abstract0_free(man, element)
    
    return output_LB, output_UB, num_out_pixels

In [151]:
def get_hidden_bounds_using_box(man, weights, biases, input_LB, input_UB, num_in_pixels):
    '''
    This function calculates the bounds of a ReLU operation followed by a hidden layer. 
    INPUT:
        - man: pointer to elina manager
        - weights: weights of the hidden layer
        - biases: biases of the hidden layer
        - input_LB: lower bound of the inputs to the hidden layer
        - input_UB: upper bound of the inputs to the hidden layer
        - num_in_pixels: number of inputs to the input layer
    
    OUTPUT:
        - output_LB: lower bound of the outputs from hidden layer
        - output_UB: upper bound of the outputs from hidden layer
        - num_out_pixels: number of outputs of hidden layer
    '''
    itv = elina_interval_array_alloc(num_in_pixels)

    ## Populate the interval
    for i in range(num_in_pixels):
        elina_interval_set_double(itv[i], input_LB[i], input_UB[i])

    ## construct input abstraction
    element = elina_abstract0_of_box(man, 0, num_in_pixels, itv)
    elina_interval_array_free(itv, num_in_pixels)
    
    # ------------------------------------------------------------------
    # Handle Affine Layer
    # ------------------------------------------------------------------

    # calculate number of outputs
    num_out_pixels = len(weights)
    
    print("[Network] Input pixels: " + str(num_in_pixels))
    print("[Network] Shape of weights: " + str(np.shape(weights)))
    print("[Network] Shape of biases: " + str(np.shape(biases)))
    print("[Network] Out pixels: " + str(num_out_pixels))

    # Create number of neurons in the layer and populate it
    # with the number of inputs to each neuron in the layer
    dimadd = elina_dimchange_alloc(0, num_out_pixels)    
    for i in range(num_out_pixels):
        dimadd.contents.dim[i] = num_in_pixels

    # Add dimensions to an ElinaAbstract0 pointer i.e. element
    elina_abstract0_add_dimensions(man, True, element, dimadd, False)
    elina_dimchange_free(dimadd)

    # Create the linear expression associated each neuron
    var = num_in_pixels
    for i in range(num_out_pixels):
        tdim = ElinaDim(var)
        linexpr0 = generate_linexpr0(weights[i], biases[i], num_in_pixels)
        # Parallel assignment of several dimensions of an ElinaAbstract0 by using an ElinaLinexpr0Array
        element = elina_abstract0_assign_linexpr_array(man, True, element, tdim, linexpr0, 1, None)
        var += 1

    # Pointer to which semantics we want to follow.
    dimrem = elina_dimchange_alloc(0, num_in_pixels)
    for i in range(num_in_pixels):
        dimrem.contents.dim[i] = i
        
    # Remove dimensions from an ElinaAbstract0
    elina_abstract0_remove_dimensions(man, True, element, dimrem)
    elina_dimchange_free(dimrem)
    
    # get bounds for each output neuron
    bounds = elina_abstract0_to_box(man,element)
    
    output_LB = np.zeros((num_out_pixels, 1), float)
    output_UB = np.zeros((num_out_pixels, 1), float)
    for j in range(num_out_pixels):
        output_LB[j] = bounds[j].contents.inf.contents.val.dbl
        output_UB[j] = bounds[j].contents.sup.contents.val.dbl
    
    
    # free out the memory allocations
    elina_interval_array_free(bounds, num_out_pixels)
    elina_abstract0_free(man, element)
    
    return output_LB, output_UB, num_out_pixels

# Linear

In [241]:
def get_bounds_using_linear(weights, biases, prev_lb, prev_ub):
    """
    RETURN: 
            - out_lb: Array of output lower bounds
            - out_ub: Array of output upper bounds
            - n_out: dimension of output
    """
    m = Model("LP")
    # Preprocessing
    prev_lb, prev_ub = prev_lb.squeeze(), prev_ub.squeeze()
    
    nontriv_relu, output_LB, output_UB, lamda_linear, mu_linear = relu_operation(input_LB, input_UB, num_in_pixels)
    
    # Input output size
    n_in, n_out = prev_lb.shape[0], weights.shape[1]
    out_lb, out_ub = np.zeros(n_out), np.zeros(n_out)
    lambdas_, mus_ = np.zeros_like(prev_ub), np.zeros_like(prev_ub)
    
    # Create lambda and mus
    row = 0 
    for (ub, lb) in zip(prev_ub, prev_lb):
        lambdas_[row] = ub / (ub - lb)
        mus_[row] = -(lb * ub) / (ub - lb)
        row +=1
    
    # Create input Variable and constrains
    x = m.addVars(n_in, lb=prev_lb, ub=prev_ub, vtype=GRB.CONTINUOUS)
    
    # Create Relu Output variable
    relu_out = m.addVars(n_in, vtype=GRB.CONTINUOUS)
    # Create Relu Output constrain
    m.addConstrs(relu_out[j] >= 0 for j in range(len(relu_out)))
    m.addConstrs(relu_out[j] >= x[j] for j in range(len(relu_out)))
    m.addConstrs(relu_out[j] <= lambdas_[j] * x[j] + mus_[j] for j in range(len(relu_out)))
    
    
    for i_out in range(n_out):
        obj = LinExpr()
        obj += biases[i_out]
        for j_out in range(weights.shape[1]):
            obj +=  relu_out[i_out]*weights[i_out,j_out]
        
        # Find Lower Bound
        m.setObjective(obj, GRB.MINIMIZE)
        m.optimize()
        out_lb[i_out] = m.objVal
        m.reset()
        # Find Upper Bound
        m.setObjective(obj, GRB.MAXIMIZE)
        m.optimize()
        out_ub[i_out] = m.objVal
        m.reset()
    
#     for idx in range(len(out)):
#         m.update() # If you have unstaged changes in the model
#         max_copy = m.copy()
#         max_copy.setObjective(out[idx], GRB.MAXIMIZE)
#         max_copy.optimize()
        
#         for v in enumerate(max_copy.getVars()):
#             if v.varName == "out":
#                 out_ub[idx] = v.x
#                 print("Obj:", max_copy.objVal)    
                
#         min_copy = m.copy()
#         min_copy.setObjective(out[idx], GRB.MINIMIZE)
#         min_copy.optimize()
        
#         for v in enumerate(min_copy.getVars()):
#             if v.varName == "out":
#                 out_lb[idx] = v.x
#                 print("Obj:", min_copy.objVal)   
            
            
    return out_lb, out_ub, len(out_ub)

# Define operations on abstract domain using linear approximation

In [242]:
def relu_operation(input_LB, input_UB, num_in_pixels):
    """
    This function computes ReLU.
    INPUT
        - input_LB: lower bound of the inputs to the ReLU
        - input_UB: upper bound of the inputs to the ReLU
        - num_in_pixels: number of inputs to ReLU    
    OUTPUT:
        - nontriv_relu: boolean array for locations where ReLU operation is non-trivial
        - output_LB: lower bound of outputs 
        - output_UB: upper bound of outputs
        - lamda_linear: slope of linear approximator line when ReLU non-trivial
        - mu_linear: y-intercept of linear approximator line when ReLU non-trivial
    
    """
    
    output_LB = np.zeros((num_in_pixels, 1), float)
    output_UB = np.zeros((num_in_pixels, 1), float)
    lambda_linear = np.zeros((num_in_pixels, 1), float)
    mu_linear = np.zeros((num_in_pixels, 1), float)
    nontriv_relu = np.zeros((num_in_pixels, 1), bool)

    for j in range(num_in_pixels):
        u = input_UB[j]
        l = input_LB[j]
        if u <= 0:
            output_LB[j] = 0
            output_UB[j] = 0
        elif l >= 0:
            output_LB[j] = l
            output_UB[j] = u
        else:
            nontriv_relu[j] = True
            lambda_linear[j] = u / (u - l)
            mu_linear[j] = - lambda_ * l / 2

    return nontriv_relu, output_LB, output_UB, lamda_linear, mu_linear

In [243]:
def get_relu_hidden_bounds_using_linear(weights, biases, input_LB, input_UB, num_in_pixels):
    '''
    This function calculates the bounds of a ReLU operation followed by a hidden layer. 
    INPUT:
        - man: pointer to elina manager
        - weights: weights of the hidden layer
        - biases: biases of the hidden layer
        - input_LB: lower bound of the inputs to the ReLU
        - input_UB: upper bound of the inputs to the ReLU
        - num_in_pixels: number of inputs to ReLU
    
    OUTPUT:
        - output_LB: lower bound of the outputs from hidden layer
        - output_UB: upper bound of the outputs from hidden layer
        - num_out_pixels: number of outputs of hidden layer
    '''
    
    # Perform the ReLU operation
    relu_nontriv, relu_LB, relu_UB, lamda_linear, mu_linear = relu_operation(input_LB, input_UB, num_in_pixels)
    
    # Create linear programming model 
    m = model("LP")
    
    # Add 
    for j in range(num_in_pixels):
        if relu_nontriv[j] == False:
            
    
    # Create input Variable and constrains
    relu_in = m.addVars(n_in, lb=prev_lb, ub=prev_ub, vtype=GRB.CONTINUOUS)
    
    # Create Relu Output variable
    relu_out = m.addVars(n_in, vtype=GRB.CONTINUOUS)
    # Create Relu Output constrain
    m.addConstrs(relu_out

    
    m.addConstrs(
    
    for bound in previous_bounds:
        m = addReluConstr(m, bound[0], bound[0])
    
    # Iterate over output neurons
    for weight, bias in zip(weights, biases):

IndentationError: expected an indented block (<ipython-input-243-cc9c5bb13feb>, line 30)

# Initialize the problem variables

In [244]:
netname = '/home/riai2018/mnist_nets/mnist_relu_3_10.txt'
specname = '/home/riai2018/mnist_images/img1.txt'
epsilon = 0.001

In [245]:
with open(netname, 'r') as netfile:
    netstring = netfile.read()
with open(specname, 'r') as specfile:
    specstring = specfile.read()
nn = parse_net(netstring)
x0_low, x0_high = parse_spec(specstring)
LB_N0, UB_N0 = get_perturbed_image(x0_low,0)

In [246]:
numlayer = nn.numlayer 

for layerno in range(numlayer):
    if(nn.layertypes[layerno] in ['ReLU', 'Affine']):
        print(nn.layertypes[layerno])

ReLU
ReLU
ReLU


# Get perturbed label (provided prediction for unperturbed is true)

In [247]:
label, _ = analyze(nn,LB_N0,UB_N0,0) # Get label of unperturbed image, i.e. eps=0

if(label == int(x0_low[0])):
    LB_N0, UB_N0 = get_perturbed_image(x0_low,epsilon)
else:
    print("image not correctly classified by the network. expected label ",int(x0_low[0]), " classified label: ", label)

# Start Verification

### 1) Define element for the input 

In [248]:
num_pixels = len(LB_N0)
numlayer = nn.numlayer 
man = elina_box_manager_alloc()

### 2) Iterate over each layer in the network and define the neural network function

In [249]:
nn.ffn_counter = 0
input_LB = LB_N0
input_UB = UB_N0
num_in_pixels = num_pixels

for layerno in range(numlayer):
    print("Layer Number: " + str(layerno))
    if(nn.layertypes[layerno] in ['ReLU', 'Affine']):
        # read the layer weights and biases
        weights = nn.weights[nn.ffn_counter]
        biases = nn.biases[nn.ffn_counter]
        np.ascontiguousarray(weights, dtype=np.double)
        np.ascontiguousarray(biases, dtype=np.double)
        
        # only hidden layer
        if (layerno == 0):
            print("HIDDEN!")
            output_LB, output_UB, num_out_pixels = get_hidden_bounds_using_box(man, weights, biases, input_LB, input_UB, num_in_pixels)
        # ReLU + hidden layer
        else:
            print("RELU + HIDDEN!")
            use_LP = True
            if not use_LP:
                output_LB, output_UB, num_out_pixels = get_relu_hidden_bounds_using_box(man, weights, biases, input_LB, input_UB, num_in_pixels)
            else:
                output_LB, output_UB, num_out_pixels = get_bounds_using_linear(weights, biases, input_LB, input_UB)

        # Iterate to next layer
        input_LB = output_LB
        input_UB = output_UB
        num_in_pixels = num_out_pixels        
        nn.ffn_counter+=1 
        
        print('---------------')

        # only ReLU layer
        if(layerno + 1 == numlayer):
            print("RELU!")
            output_LB, output_UB, num_out_pixels = get_relu_bounds_using_box(man, input_LB, input_UB, num_in_pixels)
            print('---------------')
            
    else:
        print(' net type not supported')

Layer Number: 0
HIDDEN!
[Network] Input pixels: 784
[Network] Shape of weights: (10, 784)
[Network] Shape of biases: (10,)
[Network] Out pixels: 10
---------------
Layer Number: 1
RELU + HIDDEN!
Optimize a model with 30 rows, 20 columns and 50 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [5e-01, 5e-01]
  Bounds range     [2e-01, 2e+01]
  RHS range        [6e-01, 2e+03]
Presolve removed 30 rows and 20 columns
Presolve time: 0.03s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -3.5238507e+00   0.000000e+00   0.000000e+00      1s

Solved in 0 iterations and 0.04 seconds
Optimal objective -3.523850703e+00
Optimize a model with 30 rows, 20 columns and 50 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [5e-01, 5e-01]
  Bounds range     [2e-01, 2e+01]
  RHS range        [6e-01, 2e+03]
Presolve removed 30 rows and 20 columns
Presolve time: 0.03s
Pres

  RHS range        [6e-01, 2e+03]
Presolve removed 30 rows and 20 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -4.3956196e+00   0.000000e+00   0.000000e+00      1s

Solved in 0 iterations and 0.02 seconds
Optimal objective -4.395619636e+00
Optimize a model with 30 rows, 20 columns and 50 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [4e-01, 4e-01]
  Bounds range     [2e-01, 2e+01]
  RHS range        [6e-01, 2e+03]
Presolve removed 30 rows and 20 columns
Presolve time: 0.03s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.7194050e-01   0.000000e+00   0.000000e+00      1s

Solved in 0 iterations and 0.03 seconds
Optimal objective  2.719404996e-01
Optimize a model with 30 rows, 20 columns and 50 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [4e-0

/home/riai2018/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in double_scalars
/home/riai2018/.local/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in double_scalars


GurobiError: Element 0 of a double array is Nan or Inf.

In [250]:
%debug

> /home/riai2018/analyzer/model.pxi(2846)gurobipy.Model.__addConstr()

ipdb> u
> /home/riai2018/analyzer/model.pxi(2982)gurobipy.Model.addConstr()

ipdb> u
> /home/riai2018/analyzer/model.pxi(3072)gurobipy.Model.addConstrs()

ipdb> u
> <ipython-input-241-22fac5c0c364>(31)get_bounds_using_linear()
     29     m.addConstrs(relu_out[j] >= 0 for j in range(len(relu_out)))
     30     m.addConstrs(relu_out[j] >= x[j] for j in range(len(relu_out)))
---> 31     m.addConstrs(relu_out[j] <= lambdas_[j] * x[j] + mus_[j] for j in range(len(relu_out)))
     32 
     33 

ipdb> mus_[j]
*** NameError: name 'j' is not defined
ipdb> i
*** NameError: name 'i' is not defined
ipdb> mus_
array([-3.21399744e+14,            -inf,            -inf,            -inf,
       -1.00730465e+15,            -inf, -2.38535152e+14, -4.35080870e+15,
                  -inf, -1.92974889e+15])
ipdb> q


## Check the verifiability of the network

In [67]:
# if epsilon is zero, try to classify else verify robustness 
verified_flag = True
predicted_label = 0
if(LB_N0[0]==UB_N0[0]):
    for i in range(num_out_pixels):
        inf = output_LB[i]
        flag = True
        for j in range(num_out_pixels):
            if(j!=i):
                sup = output_UB[j]
                if(inf<=sup):
                    flag = False
                    break
        if(flag):
            predicted_label = i
            break    
else:
    inf = output_LB[label]
    for j in range(num_out_pixels):
        if(j!=label):
            sup = output_UB[j]
            if(inf<=sup):
                predicted_label = label
                verified_flag = False
                break

elina_manager_free(man)   

# Check status 

In [56]:
if(verified_flag):
    print("verified")
else:
    print("can not be verified")  

verified
